In [8]:
import os
import pandas as pd
import shutil
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def make_dataframe(sdir):
    
    filepaths=[]
    labels=[]
    classlist=sorted(os.listdir(sdir) )     
    for klass in classlist:
        classpath=os.path.join(sdir, klass) 
        if os.path.isdir(classpath):
            flist=sorted(os.listdir(classpath)) 
            desc=f'{klass:25s}'
            for f in tqdm(flist, ncols=130,desc=desc, unit='files', colour='blue'):
                fpath=os.path.join(classpath,f)
                filepaths.append(fpath)
                labels.append(klass)
    Fseries=pd.Series(filepaths, name='filepaths')
    Lseries=pd.Series(labels, name='labels')
    df=pd.concat([Fseries, Lseries], axis=1) 
   
    return df

def make_and_store_images(df, augdir, n,  img_size,  color_mode='rgb', save_prefix='aug-',save_format='jpg'):
       df=df.copy()        
    if os.path.isdir(augdir):
        shutil.rmtree(augdir)
    os.mkdir(augdir)      
    for label in df['labels'].unique():    
        classpath=os.path.join(augdir,label)    
        os.mkdir(classpath) 
    total=0
   
   gen = ImageDataGenerator(rescale=1./255, rotation_range =15,                  
                     fill_mode = 'nearest', shear_range=0.2, 
                     zoom_range=0.2, width_shift_range = 0.2, 
                     height_shift_range = 0.2, horizontal_flip = True,
                     brightness_range=[0.5, 1.5])
    
    groups= df.groupby('labels') 
    for label in df['labels'].unique():  
        classdir=os.path.join(augdir, label)
        group=groups.get_group(label)   
        sample_count=len(group)   
        if sample_count == n: 
            aug_img_count = 0
            delta = n + sample_count           
            msg='{0:40s} for class {1:^30s} creating {2:^5s} augmented images'.format(' ', label, str(delta))
            print(msg, '\r', end='') 
            aug_gen=gen.flow_from_dataframe( group,  x_col='filepaths', y_col=None, target_size=img_size,
                                            class_mode=None, batch_size=1, shuffle=False, 
                                            save_to_dir=classdir, save_prefix=save_prefix, color_mode=color_mode,
                                            save_format=save_format)
            while aug_img_count < delta:
                images=next(aug_gen)            
                aug_img_count += len(images)
            total += aug_img_count        
    print('Total Augmented images created= ', total)

In [9]:
sdir=r'C:/Users/DELL/Desktop/main/license plate'
df=make_dataframe(sdir)
print (df.head())
print ('length of dataframe is ',len(df))

augdir=r'C:/Users/DELL/Desktop/main/new' 
n=7 
img_size=(224,224) 
make_and_store_images(df, augdir, n,  img_size,  color_mode='rgb', save_prefix='aug-',save_format='jpg')

telugu                   : 100%|█████████████████████████████████████████████████████████████████████████| 7/7 [00:00<?, ?files/s]

                                           filepaths   labels
0  C:/Users/DELL/Desktop/main/license plate\benga...  bengali
1  C:/Users/DELL/Desktop/main/license plate\benga...  bengali
2  C:/Users/DELL/Desktop/main/license plate\benga...  bengali
3  C:/Users/DELL/Desktop/main/license plate\benga...  bengali
4  C:/Users/DELL/Desktop/main/license plate\benga...  bengali
length of dataframe is  35
Found 7 validated image filenames.


Found 7 validated image filenames.       for class             hindi              creating  14   augmented images 
Found 7 validated image filenames.       for class            kannada             creating  14   augmented images 
Found 7 validated image filenames.       for class             tamil              creating  14   augmented images 
Found 7 validated image filenames.       for class             telugu             creating  14   augmented images 
Total Augmented images created=  70
